In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Задайте максимальное количество столбцов для вывода
pd.set_option('display.max_columns', None)

# Задайте максимальную ширину вывода для каждого столбца
pd.set_option('display.max_colwidth', None)

# Генерация данных
np.random.seed(0)

N_years = 5
days = 365 * N_years
sku_count = 10

# 8.8
koef = 8.8




# Задаем веса для SKU согласно принципу Парето
pareto_weights = (np.arange(sku_count*8, 0, -8) ** -1.5)
# pareto_weights = (np.arange(sku_count, 0, -1) ** -1.5)
# for i in range(len(pareto_weights)):
#     pareto_weights[i] += 0.2
# pareto_weights = [weight + 0.2 for weight in pareto_weights_new]
# pareto_weights = np.array([weight + 0.2 for weight in pareto_weights_new])


pareto_weights /= pareto_weights.sum()
# [0.01584834 0.0185618  0.02214873 0.02706056 0.0341002  0.04482588 0.06264608 0.09644994 0.17718986 0.5011686 ]


# for i in range(len(pareto_weights)):
#     if i == 0:
#         pareto_weights[i] += 1000


for i in range(len(pareto_weights)):
    pareto_weights[i] /= 3
# [0.00158483 0.00185618 0.00221487 0.00270606 0.00341002 0.00448259 0.00626461 0.00964499 0.01771899 0.05011686]
# print(pareto_weights)
# [0.01584834 0.0185618  0.02214873 0.02706056 0.0341002  0.04482588 0.06264608 0.09644994 0.17718986 0.5011686 ]
# [0.51584834 0.5185618  0.52214873 0.52706056 0.5341002  0.54482588 0.56264608 0.59644994 0.67718986 1.0011686 ]
# [10.01584834  0.0185618   0.02214873  0.02706056  0.0341002   0.04482588 0.06264608  0.09644994  0.17718986  0.5011686 ]
# [0.1899093  0.16912702 0.14857482 0.12827856 0.10827111 0.08859582 0.0693126  0.05051026 0.03233547 0.01508503]





def calculate_influence(current_sku, other_sku, num_v):
    if num_v == 'min_v':
        return ((current_sku/other_sku) * 0.6) * koef
    else:
        x = (0.6 * (current_sku - 10 / 100)) * koef
        if x < 0:
            return x*(-1)
        return (0.6 * (current_sku - 10 / 100))* koef


all_sku = pareto_weights

# Пройдемся по всем SKU и сравним их
def influence():
    influences = [[] for _ in range(len(all_sku))]
    for i, current_sku in enumerate(all_sku):
        for j, other_sku in enumerate(all_sku):
            if current_sku != other_sku:
                if current_sku < other_sku:
                    influence = calculate_influence(current_sku, other_sku, 'min_v')
                    influences[i].append(f"sku_{j+1}_{i+1}: {influence}")
                else:
                    influence = calculate_influence(other_sku, current_sku, 'max_v')
                    influences[i].append(f"sku_{j+1}_{i+1}: {influence}")
    return influences

# Получаем значения влияния в виде вложенного списка с ключами
influences = influence()









# def generate_promo_intervals():
#     starts = np.sort(np.random.choice(np.arange(1, days - 15), size=15, replace=False))
#     return [(start, start + 14) for start in starts]


def generate_promo_intervals():
    new_holiday = []
    starts = np.sort(np.random.choice(np.arange(1, 365 - 15), size=5, replace=False))
    tuple_date = [(start, start + 14) for start in starts]
    for i in range(N_years):
        offset = i * 365
        for key, value in tuple_date:

            new_typle=(key+offset, value+offset)
            new_holiday.append(new_typle)

    return new_holiday

promo_intervals_per_sku = {sku: generate_promo_intervals() for sku in range(1, sku_count + 1)}
# print(promo_intervals_per_sku)
data = {
    'Day': np.tile(np.arange(1, days + 1), sku_count),
    'SKU': np.repeat(np.arange(1, sku_count + 1), days),
    'Price': np.random.uniform(10, 200, size=days * sku_count),
    'Regular Sales': np.abs(np.random.normal(100, 5, days * sku_count) * np.repeat(pareto_weights, days)),
    'Promo Sales': np.zeros(days * sku_count),
    'Stock': np.random.randint(50, 200, size=days * sku_count)
}

df = pd.DataFrame(data)



new_dicts = []


data = promo_intervals_per_sku

# Создание нового словаря с замененными ключами
new_data = {f"sku_{key}": value for key, value in data.items()}


# Создание списка ключей из исходного словаря
keys = list(data.keys())

# Создание списка словарей без одной строки
#  new_dicts  список интервалов на каждой строке sku*sku
new_dicts = []
d = 0
for i in range(len(keys)):
    d+=1
    new_dict = {f"sku_{key}_{d}": value for key, value in data.items() if key != keys[i]}
    new_dicts.append(new_dict)






# Недельная сезонность
недельные_коэффициенты = {
    0: 0.9,  # Понедельник
    1: 1.0,  # Вторник
    2: 1.0,  # Среда
    3: 1.1,  # Четверг
    4: 1.3,  # Пятница
    5: 1.5,  # Суббота
    6: 1.4   # Воскресенье
}
# Месячная сезонность
# month_coefficients = {
#     1: 1.2, 2: 1.1, 3: 1.0, 4: 0.9, 5: 0.9,
#     6: 0.9, 7: 1.0, 8: 1.0, 9: 1.1, 10: 1.1,
#     11: 1.2, 12: 1.3, 13: 1.2, 14: 1.1, 15: 1.0, 16: 0.9, 17: 0.9,
#     18: 0.9, 19: 1.0, 20: 1.0, 21: 1.1, 22: 1.1,
#     23: 1.2, 24: 1.3,
#     25: 1.2, 26: 1.1, 27: 1.0, 28: 0.9, 29: 0.9, 30: 0.9,
#     31: 1.0, 32: 1.0, 33: 1.1, 34: 1.1, 35: 1.2, 36: 1.3
# }

month_coefficients = {
    1: 1.2, 2: 1.1, 3: 1.0, 4: 0.9, 5: 0.9,
    6: 0.9, 7: 1.0, 8: 1.0, 9: 1.1, 10: 1.1,
    11: 1.2, 12: 1.3
}
месячные_коэффициенты = {}

for i in range(N_years):
    offset_yeas = i * 12
    for num, indx in month_coefficients.items():
        new = num+offset_yeas
        месячные_коэффициенты[new]= indx


# Праздничная сезонность
# праздничные_коэффициенты = {
#     1: 1.5,   # Новый год
#     45: 1.2,  # День влюбленных
#     95: 1.3,  # Пасха
#     196: 1.4, # Летние распродажи
#     333: 1.7, # Черная пятница
#     359: 1.5,  # Рождество
#     366: 1.5,   # Новый год
#     410: 1.2,  # День влюбленных
#     460: 1.3,  # Пасха
#     561: 1.4, # Летние распродажи
#     698: 1.7, # Черная пятница
#     723: 1.5,  # Рождество
#     731: 1.5,   # Новый год
#     775: 1.2,  # День влюбленных
#     825: 1.3,  # Пасха
#     926: 1.4, # Летние распродажи
#     1063: 1.7, # Черная пятница
#     1089: 1.5,  # Рождество
# }

holiday_coefficients = {
    1: 1.5,   # Новый год
    7: 1.5,   # Рождество
    45: 1.2,  # День влюбленных
    95: 1.3,  # Пасха
    196: 1.4, # Летние распродажи
    333: 1.7, # Черная пятница
    365: 1.5,  # Рождество
}

праздничные_коэффициенты = {}

for i in range(N_years):
    offset = i * 365
    for key, value in holiday_coefficients.items():
        new_key = key + offset
        праздничные_коэффициенты[new_key] = value




def holiday_multiplier(day):

    for holiday, boost in праздничные_коэффициенты.items():
        days_before_holiday = holiday - day
        if 0 <= days_before_holiday <= 6:
            return 1 + (boost - 1) * (1 - days_before_holiday / 7)
    return 1

df['Weekday'] = df['Day'] % 7

df['Seasonality'] = df['Weekday'].map(недельные_коэффициенты)
df['Month'] = ((df['Day'] - 1) // 30) + 1
df['Monthly Seasonality'] = df['Month'].map(месячные_коэффициенты)
df['Holiday Boost'] = df['Day'].apply(holiday_multiplier)

df['Monthly Seasonality'].fillna(value=1, inplace=True)

df['Regular Sales'] *= df['Seasonality'] * df['Monthly Seasonality'] * df['Holiday Boost']
# nan_count = df['Monthly Seasonality'].isna().sum()

# print("Number of NaN values in 'Monthly Seasonality':", nan_count)

# nan_count_h = df['Holiday Boost'].isna().sum()

# print("Number of NaN values in 'Holiday Boost':", nan_count_h)
# nan_count_q = df['Seasonality'].isna().sum()

# print("Number of NaN values in 'Seasonality':", nan_count_q)




for sku, intervals in promo_intervals_per_sku.items():

    for start, end in intervals:


        mask = (df['SKU'] == sku) & (df['Day'] >= start) & (df['Day'] <= end)

        df.loc[mask, 'Price'] *= np.random.uniform(0.7, 0.8)
        promo_increase = np.abs(np.random.normal(200, 5, np.sum(mask)) * pareto_weights[sku-1])
        df.loc[mask, 'Promo Sales'] = promo_increase
        df.loc[mask, 'Regular Sales'] = 0

        # other_skus_mask = (df['SKU'] != sku) & (df['Day'] >= start) & (df['Day'] <= end)
        # df.loc[other_skus_mask, 'Regular Sales'] *= np.random.uniform(0.7, 0.9)


df['index_promo'] = ''

# Iterate over the 'data' dictionary
for dos in new_dicts:
    for sku, ranges in dos.items():
        for start, end in ranges:
            # Expand the range and check if any value matches the 'DAYS' column
            matching_days = range(start, end + 1)
            df.loc[df['Day'].isin(matching_days), 'index_promo'] = sku

for sublist in influences:

    for item in sublist:
        key, value = item.split(': ')
        df.loc[df['index_promo'] == key, 'res_index_promo'] = float(value)
# замена nan на 1 чтобы умножить столбцы без изменений
df['res_index_promo'].fillna(value=1, inplace=True)
# срез максимальных индексов 286 позиция слишком улетела 4 значение
# df['res_index_promo'] = df['res_index_promo'].apply(lambda x: 1.7 if x > 1.7 else x)
# показ результатов полученных индексов
df['result_1'] = df['Regular Sales'] * df['res_index_promo']
# регулярные индексы с связью sku*sku


df['Regular Sales'] = df['Regular Sales'] * df['res_index_promo']
df['Regular_Sales_Promo_Sales'] = df['Regular Sales'] + df['Promo Sales']


# print(df.loc[185:200])
# index_search = df[df['SKU'] == 10].index[0]
# print(index_search)
# result = (df['SKU'] == 1).all()

# count_zeros_promo = (df['Promo Sales'] == 0).sum()

# print(count_zeros_promo,'count_zeros_promo')  # Вывод количества нулей

# count_zeros_regul = (df['Regular Sales'] == 0).sum()

# print(count_zeros_regul,'count_zeros_Regular Sales')  # Вывод количества нулей
# print(sum([count_zeros_promo,count_zeros_regul]), 'сумма нулей')
# print(df.shape[0], 'total _df')





# Создание модели

# print(df.loc[30:50,'Holiday Boost'])



for sku in range(1, sku_count + 1):
    plt.figure(figsize=(22, 6))
    subset = df[df['SKU'] == sku]
    plt.plot(subset['Day'], subset['Regular Sales'], label='Regular Sales')
    plt.plot(subset['Day'], subset['Promo Sales'], label='Promo Sales', color='red')
    plt.title(f"Sales for SKU {sku}")
    plt.legend()


    # Изменение шага на оси x
    plt.xticks(range(0, days, 50))
    # праздничные_коэффициенты
    # Выделение интервалов зеленым цветом
     # Выделение интервалов зеленым цветом
    interval_colors = ['green'] * sku_count
    bound_pr = []
    bound_ol = []
    for start in праздничные_коэффициенты:
        bound_ol.append(start)
        if start != 1:
            end = start - 6
            bound_pr.append((start,end))
            plt.axvspan(start, end, facecolor=interval_colors[sku-1], alpha=0.3)



    # interval_colors = ['yellow'] * sku_count
    # for intervals in new_dicts:
    #     for sku_indx, tuple_num in intervals.items():

    #         for start, end in tuple_num:
    #             plt.axvspan(start, end, facecolor=interval_colors[sku-1], alpha=0.3)

    interval_colors = ['yellow'] * sku_count

    for sku_indx, tuple_num in new_dicts[sku-1].items():

        for start, end in tuple_num:
            plt.axvspan(start, end, facecolor=interval_colors[sku-1], alpha=0.3)





plt.show()
# print(bound_pr,'green')
# print(bound_ol, 'ol')
# c = df[df['Holiday Boost']>1]['Day'].tolist()
# print(c)
# print(df.loc[-10:10950])


# Суммарные продажи по SKU
total_sales_per_sku = df.groupby('SKU')[['Regular Sales', 'Promo Sales']].sum().sum(axis=1)
plt.figure(figsize=(12, 6))
total_sales_per_sku.plot(kind='bar')
plt.title("Total Sales for Each SKU")
plt.ylabel("Total Sales")
plt.xlabel("SKU")
plt.show()






# Добавим столбец 'Date', чтобы использовать его для группировки по неделям
df['Date'] = pd.to_datetime(df['Day'], origin='2023-01-01', unit='D')

# Группируем по SKU и неделям, затем суммируем продажи
weekly_sales = df.groupby('SKU').apply(lambda group: group.set_index('Date').resample('W').sum())

# Отрисовка графиков недельных продаж для каждого SKU
for sku in range(1, sku_count + 1):
    plt.figure(figsize=(22, 6))

    # Выбор нужного SKU из отсортированных данных
    subset = weekly_sales.loc[sku]

    # Рисуем график недельных продаж
    plt.plot(subset.index, subset['Regular Sales'] + subset['Promo Sales'], label='Total Weekly Sales')
    plt.title(f"Weekly Sales for SKU {sku}")
    plt.ylabel("Sales")
    plt.xlabel("Week")
    plt.legend()
    plt.grid(True)
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from google.colab import output
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split  # Добавление необходимых библиотек

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error
from google.colab import files


if 'COLAB_TPU_ADDR' not in os.environ:
    print('NOT connected to a TPU')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print('Connected to TPU: ', tpu_address)
  # Инициализация TPU
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Number of replicas:', strategy.num_replicas_in_sync)

# Задайте максимальное количество столбцов для вывода
pd.set_option('display.max_columns', None)

# Задайте максимальную ширину вывода для каждого столбца
pd.set_option('display.max_colwidth', None)

# Индексы строк с пропущенными значениями
df.copy()

epochs = 150
sequence_length = 100
forecast_days = 365
# sku_count = 1


indx_len = 1095
sku_mse = []
sku_mse_year =[]






for sku in range(1, sku_count + 1):
    '''X_test (200, 100, 3)
Y_test (200, 365)'''
    # if sku !=9:
    #     continue
    sale = df[df['SKU'] == sku].head(1460)
    sales = sale.copy()

    five_year_test = df[df['SKU'] == sku].tail(365)
    five_year_test_X = five_year_test.copy()
    # if sku == 9:
    #             # предположим, что у вас есть DataFrame df
    #     sales.to_csv('df.csv')
    #     files.download('df.csv')



        # Предварительная обработка данных
    sales['Regular_Sales_Promo_Sales'] = (sales['Regular_Sales_Promo_Sales'] - sales['Regular_Sales_Promo_Sales'].min()) / (sales['Regular_Sales_Promo_Sales'].max() - sales['Regular_Sales_Promo_Sales'].min())
    sales['Price'] = (sales['Price'] - sales['Price'].min()) / (sales['Price'].max() - sales['Price'].min())
    sales['Stock'] = (sales['Stock'] - sales['Stock'].min()) / (sales['Stock'].max() - sales['Stock'].min())

       # Предварительная обработка данных для 5 года
    five_year_test_X['Regular_Sales_Promo_Sales'] = (five_year_test_X['Regular_Sales_Promo_Sales'] - sale['Regular_Sales_Promo_Sales'].min()) / (sale['Regular_Sales_Promo_Sales'].max() - sale['Regular_Sales_Promo_Sales'].min())
    five_year_test_X['Price'] = (five_year_test_X['Price'] - sale['Price'].min()) / (sale['Price'].max() - sale['Price'].min())
    five_year_test_X['Stock'] = (five_year_test_X['Stock'] - sale['Stock'].min()) / (sale['Stock'].max() - sale['Stock'].min())

    X_train = []
    y_train = []


    X_train_five_year = []

    for i in range(len(sales) - sequence_length - forecast_days + 1):
        X = sales[['Regular_Sales_Promo_Sales', 'Price', 'Stock']].values[i: i + sequence_length]

        X_train.append(X)
        y_train.append(sales['Regular_Sales_Promo_Sales'].values[i + sequence_length: i + sequence_length + forecast_days])

    for five in range(len(five_year_test_X)- sequence_length +1):
        X_year = five_year_test_X[['Regular_Sales_Promo_Sales', 'Price', 'Stock']].values[five: five + sequence_length]
        X_train_five_year.append(X_year)



    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_train_five_year = np.array(X_train_five_year)



    # X_train = X_train.reshape(-1, sequence_length, 1)  # Решейпинг для множества временных рядов

    # Использование strategy.scope для обучения модели на TPU
    with strategy.scope():


        model = Sequential([
            LSTM(64, input_shape=(sequence_length, 3), return_sequences=True), #return_sequences=True
            Dropout(0.2),
            LSTM(64, activation='relu'),
            Dense(forecast_days)
            ])

        model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy','mean_squared_error'])

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


    # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f'./log_Tenser_{sku}', histogram_freq=1,embeddings_freq=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True)

    model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose=0)  #tensorboard_callback

        # Проверка предсказаний
    predictions = model.predict(X_train_five_year)




    # Обратное преобразование предсказанных значений
    def inverse_transform(y, y_min, y_max):
        return y * (y_max - y_min) + y_min

    # # Обратное преобразование фактических значений
    # y_test_inv = inverse_transform(y_test, sale['Regular_Sales_Promo_Sales'].min(), sale['Regular_Sales_Promo_Sales'].max())
    # print('min2',sale['Regular_Sales_Promo_Sales'].min())
    # Обратное преобразование предсказанных значений
    predictions_inv = inverse_transform(predictions, sale['Regular_Sales_Promo_Sales'].min(), sale['Regular_Sales_Promo_Sales'].max())


    # Сравнение фактических значений с предсказанными значениями
    # comparison = pd.DataFrame({'Actual': y_test_inv.flatten(), 'Predicted': predictions_inv.flatten()})

    # comparison_0 = pd.DataFrame({'Actual': y_test_inv[0], 'Predicted': predictions_inv[0]})

    # Вычисление среднеквадратичной ошибки (MSE) между фактическими и предсказанными значениями
    # mse_flatten = mean_squared_error(comparison['Actual'], comparison['Predicted'])

    # mse_0 = mean_squared_error(comparison_0['Actual'], comparison_0['Predicted'])


    # логика на поиск по строчкам максимум MSE


    len_pred_365 = [five_year_test['Regular_Sales_Promo_Sales'].values.tolist() for _ in range(len(predictions_inv))]

    min_mse_pd = pd.DataFrame({'y_true': len_pred_365, 'predict':predictions_inv.tolist()})
    min_mse_pd['MSE'] = min_mse_pd.apply(lambda row: mean_squared_error(row['y_true'], row['predict']), axis=1)
    min_row = min_mse_pd.loc[min_mse_pd['MSE'].idxmin()]
    indx_min_mse = min_mse_pd['MSE'].idxmin()
    min_row_mse = min_row['MSE']
    min_row_predict = min_row['predict']



    min_mse_pd_0 = pd.DataFrame({'y_true': five_year_test['Regular_Sales_Promo_Sales'].values.tolist(), 'predict':predictions_inv.tolist()[0]})
    # min_mse_pd['MSE'] = min_mse_pd.apply(lambda row: mean_squared_error(row['y_true'], row['predict']), axis=1)


    # min_row = min_mse_pd.loc[min_mse_pd['MSE'].idxmin()]
    # indx_min_mse = min_mse_pd['MSE'].idxmin()
    # min_row_mse = min_row['MSE']
    # min_row_predict = min_row['predict']
    # # print(min_row)
    # max_row = min_mse_pd.loc[min_mse_pd['MSE'].idxmax()]
    # indx_max_mse = min_mse_pd['MSE'].idxmax()
    # max_row_mse = max_row['MSE']
    # max_row_predict = max_row['predict']

    # # print()
    # y_test_inv_min=y_test_inv[indx_min_mse]

    # min_mse_pd['MAE'] = min_mse_pd.apply(lambda row: mean_absolute_error(row['y_true'], row['predict']), axis=1)
    # min_row_msel_to_mse = min_mse_pd.loc[indx_min_mse,'MAE']

    # sku_mse.append([sku, mse_flatten,mse_0,min_row_mse,indx_min_mse,min_row_msel_to_mse,max_row_mse,indx_max_mse, epochs,sequence_length,forecast_days])

    # sku_mse_year.append([sku,mean_squared_error(min_mse_pd['y_true'], min_mse_pd['predict'])])




    fig = plt.figure(figsize=(22, 12))
    # fig.subplots_adjust(hspace=0.5)

    ax1 = fig.add_subplot(2, 1, 1)

    # ax1.figure(figsize=(22, 6))
    ax1.plot(range(len(sale)), sale['Regular_Sales_Promo_Sales'], label='Исходные данные')
    ax1.set_xticks(range(0, len(sale), 50))
    # plt.xlim(0, 1095)
    start  = len(sale)
    stop = start + len(predictions_inv[0])

    ax1.plot(np.arange(start, stop), five_year_test['Regular_Sales_Promo_Sales'].values.tolist(), label='Тестовые данные прогноза, y_test', color='green')  # Добавление зеленой линии
    ax1.plot(np.arange(start, stop), min_row_predict, label='Прогноз, predict', linestyle='dashed')
    # plt.xticks(np.arange(start, stop, step=50))
    # plt.xlim(start, stop)

    ax1.set_xlabel('Дни')
    ax1.set_ylabel('Regular_Sales_Promo_Sales')
    ax1.set_title(f"{forecast_days} Days Forecast for SKU {sku}")
    color_change = 999999
    start_indx = 0
    stop_indx = 0
    for _ in range(1,N_years):

        stop_indx += 365
        start_indx = stop_indx - 365
        color_change -= 123456

        ax1.axvspan(start_indx, stop_indx, facecolor=f'#{color_change}', alpha=0.5)

    # plt.axvspan(1, 365, facecolor='#CCCCCC', alpha=0.5)
    # plt.axvspan(366, 730, facecolor='#999999', alpha=0.5)
    # plt.axvspan(731, 1095, facecolor='#666666', alpha=0.5)
    # plt.axvspan(731, 1095, facecolor='#333333', alpha=0.5)

    ax1.legend()
    ax2 = fig.add_subplot(2, 1, 2)

    ax2.plot(np.arange(0, 365), five_year_test['Regular_Sales_Promo_Sales'].values.tolist(), label='Тестовые данные прогноза, y_test', color='green')  # Добавление зеленой линии
    ax2.plot(np.arange(0, 365), min_row_predict, label='Прогноз, predict', linestyle='dashed', color='#FF8C00')
    ax2.set_xlabel('Дни')
    ax2.set_ylabel('Regular_Sales_Promo_Sales')
    ax2.set_title(f"Масштабированое предсказания модели за 365 дней. SKU {sku}")

     # Выделение интервалов зеленым цветом
    interval_colors = '#777777'
    bound_pr = []
    bound_ol = []
    trimmed_dict = dict(list(праздничные_коэффициенты.items())[:7])

    for start in trimmed_dict:
        bound_ol.append(start)
        if start != 1:
            if start !=7:
                end = start - 6
                bound_pr.append((start,end))


                plt.axvspan(end,start, color=interval_colors, alpha=0.3)
            else:

                plt.axvspan(0,7, color=interval_colors, alpha=0.3,label='Праздничные дни')


    # plt.xticks(np.arange(start, stop, step=50))
    # indx_len+=1095
    sku_mse_year.append([sku,min_row_mse,mean_squared_error(min_mse_pd_0['y_true'], min_mse_pd_0['predict']),indx_min_mse])
    ax2.legend()
    print()
plt.show()

# res = pd.DataFrame(sku_mse, columns=[["   sku", f"   MSE_всех строк {predictions_inv.shape}", "    MSE_нулевого индекса", "   MSE_минимум", "   Индекс_MSE_минимума", "   MAE_минимум","   MSE_максимум", "    Индекс_MSE_максимум", "   Эпохи", "   Длина_последователностей","   Длина_предсказаний"]])
# result_string = res.to_string(index=False)
# print(result_string)

print(pd.DataFrame(sku_mse_year, columns=['  SKU','    MSE_MIN','    MSE_0', '    indx_min']))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=./log_Tenser_1